In [ ]:
# Install required libraries
!pip install --quiet pillow pandas gdown

import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import os

In [ ]:
# Create folders
os.makedirs("photos", exist_ok=True)
os.makedirs("id_cards", exist_ok=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Update the path to where your CSV file is located
csv_path = '/content/drive/My Drive/Colab Notebooks/dataset1.csv'

# Load the CSV file
df = pd.read_csv(csv_path)

# Show the first few rows
df.head()

,Timestamp,Name,Phone no.,Plot and Flat Number,Blood Group,Emergency Contact Name of local guardian,Emergency Phone no of local guardian,Allergy(if any),Home Town Address in detail,Passport size Photo (To be printed on ID card)
0,2025/06/21 8:20:00 PM GMT+5:30,Soham Udani,123456789,A 4902,A+,Jinal Udani,987654321,No,"603/A wing, Radha bldg, Malad west",https://drive.google.com/u/0/open?usp=forms_we...
1,2025/06/21 8:30:24 PM GMT+5:30,1st Test,1234567890,12 Flat 6,O,TestData 2,7890123456,pollen,2nd lane Bhilai,https://drive.google.com/u/0/open?usp=forms_we...


In [ ]:
# Helper function to convert shareable Google Drive link to direct download
def get_direct_gdrive_link(shareable_link):
    file_id = None
    if "id=" in shareable_link:
        file_id = shareable_link.split("id=")[-1]
    elif "file/d/" in shareable_link:
        file_id = shareable_link.split("/file/d/")[-1].split("/")[0]
    if file_id:
        return f"https://drive.google.com/uc?id={file_id}"
    return None

In [ ]:
# Download images
for i, row in df.iterrows():
    name = row['Name']
    gdrive_link = row['Passport size Photo (To be printed on ID card)']
    direct_link = get_direct_gdrive_link(gdrive_link)
    if direct_link:
        output_path = f'photos/{name}.jpg'
        try:
            gdown.download(direct_link, output_path, quiet=False)
        except Exception as e:
            print(f"Failed to download image for {name}: {e}")



Downloading...
From: https://drive.google.com/uc?id=1ss8PHRmJaJB6tPVBiVME43ctXSzwTdI5
To: /content/photos/Soham Udani.jpg
100%|██████████| 159k/159k [00:00<00:00, 67.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_BVVxMIw1cUhbgQyYGjkoNkUljMGR2od
To: /content/photos/1st Test .jpg
100%|██████████| 18.2k/18.2k [00:00<00:00, 31.5MB/s]


In [ ]:
print(df.columns.tolist())


['Timestamp', 'Name', 'Phone no.', 'Plot and Flat Number ', 'Blood Group', 'Emergency Contact Name of local guardian ', 'Emergency Phone no of local guardian', 'Allergy(if any)', 'Home Town Address in detail', 'Passport size Photo (To be printed on ID card)']


In [ ]:
import os

for root, dirs, files in os.walk("/usr/share/fonts"):
    for file in files:
        if file.endswith(".ttf"):
            print(os.path.join(root, file))


/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf
/usr/share/fonts/truetype/liberation/LiberationSans-Italic.ttf
/usr/share/fonts/truetype/liberation/LiberationSansNarrow-BoldItalic.ttf
/usr/share/fonts/truetype/liberation/LiberationSerif-Italic.ttf
/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf
/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf
/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Italic.ttf
/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Bold.ttf
/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf
/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf
/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf
/usr/share/fonts/truetype/liberation/LiberationSerif-Regular.ttf
/usr/share/fonts/truetype/liberation/LiberationSans-BoldItalic.ttf
/usr/share/fonts/truetype/liberation/LiberationMono-BoldItalic.ttf
/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf
/usr/share/fonts/truetyp

In [ ]:
import os
import glob
import re
import pandas as pd
import requests
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont

In [ ]:
# Setup font
font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"
font_large = ImageFont.truetype(font_path, 36)
font_small = ImageFont.truetype(font_path, 28)

# Output path
output_path = '/content/id_cards'
os.makedirs(output_path, exist_ok=True)

# 📐 Dimensions
WIDTH, HEIGHT = 600, 350
LOGO_SIZE = (80, 80)
PHOTO_SIZE = (150, 150)
LOGO_POSITION = (WIDTH - LOGO_SIZE[0] - 10, HEIGHT - LOGO_SIZE[1] - 10)

# 🖼️ Load logo
logo_path = os.path.join('/content/drive/My Drive/Colab Notebooks', 'logo.jpg')
logo = Image.open(logo_path).resize(LOGO_SIZE)

# 🔗 Function to load photo from Google Drive link
def get_photo_from_drive_url(url):
    try:
        if '/d/' in url:
            file_id = url.split('/d/')[1].split('/')[0]
        elif 'id=' in url:
            file_id = url.split('id=')[1].split('&')[0]
        else:
            return None
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
        response = requests.get(download_url)
        response.raise_for_status()
        return Image.open(BytesIO(response.content)).resize(PHOTO_SIZE)
    except Exception as e:
        print(f"⚠️ Error loading image from Google Drive: {e}")
        return None

# 🪪 Generate ID Cards
def clean_filename(name):
    return re.sub(r'[^\w\s-]', '', name).strip().replace(' ', '_')

for idx, row in df.iterrows():
    name = row['Name']
    photo_name = clean_filename(name)

    # Front Side
    front = Image.new('RGB', (WIDTH, HEIGHT), color='white')
    draw_f = ImageDraw.Draw(front)
    draw_f.text((200, 30), f"Name: {name}", fill='black', font=font_large)
    draw_f.text((200, 110), f"Phone: {row['Phone no.']}", fill='black', font=font_small)
    draw_f.text((200, 170), f"Address: {row['Plot and Flat Number ']}", fill='black', font=font_small)
    draw_f.text((200, 230), f"Blood Group: {row['Blood Group']}", fill='black', font=font_small)

    photo = get_photo_from_drive_url(row['Passport size Photo (To be printed on ID card)'])

    if not photo:
      print(f"⚠️ Photo not found or invalid for {name}, skipping ID card.")
      continue  # Skip to next person if photo isn't valid

    # Only proceed if photo is valid
    front.paste(photo, (30, 20))
    front.paste(logo, LOGO_POSITION)
    front.save(os.path.join(output_path, f"{photo_name}_ID_Front.png"))


    # Back Side
    back = Image.new('RGB', (WIDTH, HEIGHT), color='white')
    draw_b = ImageDraw.Draw(back)
    draw_b.text((30, 40), f"Emergency Contact: {row['Emergency Contact Name of local guardian ']}", fill='black', font=font_small)
    draw_b.text((30, 100), f"Emergency Number: {row['Emergency Phone no of local guardian']}", fill='black', font=font_small)
    draw_b.text((30, 160), f"Allergy: {row['Allergy(if any)']}", fill='black', font=font_small)

    back.paste(logo, LOGO_POSITION)
    back.save(os.path.join(output_path, f"{photo_name}_ID_Back.png"))

    print(f"✅ Generated ID card (front & back) for {name}")

# 📄 Generate A4 Layout Sheet
A4_WIDTH, A4_HEIGHT = 2480, 3508
ROWS, COLS = 5, 2
MARGIN_X, MARGIN_Y = 100, 100
GAP_X, GAP_Y = 50, 40

CARD_WIDTH = (A4_WIDTH - 2 * MARGIN_X - GAP_X) // 2
CARD_HEIGHT = (A4_HEIGHT - 2 * MARGIN_Y - (ROWS - 1) * GAP_Y) // ROWS

front_images = sorted(glob.glob(os.path.join(output_path, "*_ID_Front.png")))
back_images = sorted(glob.glob(os.path.join(output_path, "*_ID_Back.png")))

sheet = Image.new("RGB", (A4_WIDTH, A4_HEIGHT), "white")

def paste_card(img_path, x, y):
    if img_path:
        img = Image.open(img_path).resize((CARD_WIDTH, CARD_HEIGHT))
    else:
        img = Image.new("RGB", (CARD_WIDTH, CARD_HEIGHT), "white")
        draw = ImageDraw.Draw(img)
        draw.text((10, 10), "EMPTY", fill="gray")
    sheet.paste(img, (x, y))
    draw_sheet = ImageDraw.Draw(sheet)
    draw_sheet.rectangle([x, y, x + CARD_WIDTH, y + CARD_HEIGHT], outline="black", width=3)

# Paste cards in layout
for i in range(ROWS):
    y = MARGIN_Y + i * (CARD_HEIGHT + GAP_Y)
    front_path = front_images[i] if i < len(front_images) else None
    back_path = back_images[i] if i < len(back_images) else None

    paste_card(front_path, MARGIN_X, y)
    paste_card(back_path, MARGIN_X + CARD_WIDTH + GAP_X, y)

# Save final A4 sheet
sheet.save(os.path.join(output_path, "ID_Sheet_BlueBoxLayout.png"))
sheet.save(os.path.join(output_path, "ID_Sheet_BlueBoxLayout.pdf"))

print("🎉 ID layout sheet saved to '/content/id_cards' folder!")


✅ Generated ID card (front & back) for Soham Udani
✅ Generated ID card (front & back) for 1st Test 
🎉 ID layout sheet saved to '/content/id_cards' folder!
